In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

In [2]:
# File and folder paths
binary_dirpath = "binary-cropland-120m"
prob_dirpath = "proba-cropland-120m"

out_binary = "atlasai_croplandhistory_binary_2009_2019_120m.tif"
out_prob = "atlasai_croplandhistory_proba_2009_2019_120m.tif"

# Make a search criteria to select the DEM files
search_criteria = "atlas*.tif"

b_q = os.path.join(binary_dirpath, search_criteria)
p_q = os.path.join(prob_dirpath, search_criteria)

print(b_q, p_q)

binary-cropland-120m/atlas*.tif proba-cropland-120m/atlas*.tif


In [3]:
binary_files = glob.glob(b_q)
prob_files = glob.glob(p_q)

In [5]:
bin_files_to_mosaic = []
prob_files_to_mosaic = []

for fp in binary_files:
    bin_src = rasterio.open(fp)
    bin_files_to_mosaic.append(bin_src)

In [6]:
for fp in prob_files:
    prob_src = rasterio.open(fp)
    prob_files_to_mosaic.append(prob_src)    

In [7]:
%time prob_mosaic, prob_out_trans = merge(prob_files_to_mosaic)

CPU times: user 38.7 s, sys: 7.15 s, total: 45.9 s
Wall time: 37.2 s


In [8]:
%time bin_mosaic, bin_out_trans = merge(bin_files_to_mosaic)

CPU times: user 49.4 s, sys: 11.7 s, total: 1min 1s
Wall time: 51.7 s


In [11]:
prob_out_meta = prob_src.meta.copy()

prob_out_meta.update({"driver": "GTiff",
                "height": prob_mosaic.shape[1],
                "width": prob_mosaic.shape[2],
                "transform": prob_out_trans,
                "nodata": 0
                })

bin_out_meta = bin_src.meta.copy()

bin_out_meta.update({"driver": "GTiff",
                "height": bin_mosaic.shape[1],
                "width": bin_mosaic.shape[2],
                "transform": bin_out_trans,
                "nodata": 0
                })

In [15]:
with rasterio.open(out_prob, "w", **prob_out_meta) as dest:
    dest.write(prob_mosaic)

In [16]:
with rasterio.open(out_binary, "w", **bin_out_meta) as dest:
    dest.write(bin_mosaic)